# pytorch study

In [3]:
import torch
import numpy as np
import torch.nn as nn

> dynamic computation graph

## baisc opreation

In [4]:
x = torch.empty(5,3)
x = torch.rand(5,3)
x = torch.zeros(5,3)
x = torch.tensor([5.2,3])

x

tensor([5.2000, 3.0000])

In [5]:
x = x.new_ones(5,3)
x
x = torch.randn_like(x)
x

tensor([[ 0.1883,  0.6618, -0.5716],
        [-0.9801,  1.5696, -0.1388],
        [ 0.7079, -0.0908,  1.5254],
        [-0.2383,  0.7052, -0.4564],
        [-0.2630, -0.2869, -0.8094]])

In [7]:
y = torch.rand(5,3)

In [8]:
y.add_(x)

tensor([[ 0.8058,  1.2425, -0.2214],
        [-0.6797,  2.0575,  0.7550],
        [ 1.3288,  0.2738,  2.1069],
        [ 0.1015,  0.7158,  0.1911],
        [-0.0689, -0.0463,  0.1667]])

In [10]:
x = torch.randn(4,4)
y = x.view(16) # reshape
y
z = x.view(2,-1)
z = x.view(-1,8)
z

tensor([[-1.5212,  1.0954,  1.1619, -0.1462,  0.3967,  0.2487, -0.0570, -2.6277],
        [-0.8392, -0.6224, -0.4463,  2.0563,  0.5460, -0.2775,  0.1033, -1.2407]])

In [23]:
# dir(x)

In [11]:
a = x.numpy()
a

array([[-1.5211594 ,  1.0953665 ,  1.1619263 , -0.14624445],
       [ 0.39672455,  0.24868433, -0.05697396, -2.6276767 ],
       [-0.83921117, -0.62244636, -0.44629204,  2.05628   ],
       [ 0.5459957 , -0.27749163,  0.10329514, -1.2407198 ]],
      dtype=float32)

In [12]:
b = torch.from_numpy(a)
b

tensor([[-1.5212,  1.0954,  1.1619, -0.1462],
        [ 0.3967,  0.2487, -0.0570, -2.6277],
        [-0.8392, -0.6224, -0.4463,  2.0563],
        [ 0.5460, -0.2775,  0.1033, -1.2407]])

In [13]:
torch.cuda.is_available()

False

In [15]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    x = x.to(device) # x 移至gpu
    w = x*2
    print(w.to('cpu',torch.double))
    print(w.cpu().data.numpy())

## numpy MLP


In [16]:
N,D_in,H,D_out=64,1000,100,10
# N:features H:hidden x*w1 =>H H*w2=>D_out
x = np.random.randn(N,D_in)
y = np.random.randn(N,D_out)
w1 = np.random.randn(D_in,H)
w2 = np.random.randn(H,D_out)
learning_rate = 1e-6

In [17]:
for i in range(500):
    # forward pass
    h = x.dot(w1) #(N,H)
    h_relu = np.maximum(h,0) #relu #(N,H)
    y_pred = h_relu.dot(w2) #(N,D_out)
    
    # loss fun
    loss = np.square(y_pred-y).mean()
    print(i,loss)
    
    #backward pass gradient
    grad_y_pred = 2.0*(y_pred-y) #loss 求导
    grad_w2 = h_relu.T.dot(grad_y_pred) # h_relu(局部梯度) * grad_y_pred(上游梯度)
    grad_h_relu = grad_y_pred.dot(w2.T) # w2(局部梯度) * grad_y_pred(上游梯度)
    grad_h = grad_h_relu.copy()
    grad_h[h<0]=0
    grad_w1 = x.T.dot(grad_h) # x(局部梯度) * grad_h(上游梯度)
    #backward pass update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    

0 41325.508806827
1 34564.55487398665
2 35365.619261157335
3 38960.729022238425
4 41193.25502949688
5 38286.904072967605
6 29918.910069145284
7 19580.03355842125
8 11278.254960299437
9 6161.012870792358
10 3451.5976864330614
11 2096.3823414499357
12 1408.7687590202008
13 1037.0357285458024
14 815.6147799051084
15 669.5616214532379
16 564.2231941852039
17 483.31297720021587
18 418.40579790450954
19 364.9173547214353
20 320.0541916826304
21 281.99199007400466
22 249.45052309014153
23 221.4418611752564
24 197.24171792443278
25 176.21818154714805
26 157.8902625045445
27 141.8470166580557
28 127.75817709993484
29 115.33299900317691
30 104.36018673151372
31 94.62732300897179
32 85.95912008405423
33 78.22516780558263
34 71.30655266355332
35 65.10497406466354
36 59.530519847669076
37 54.50796241209944
38 49.974439030121616
39 45.880404118991265
40 42.18096723464423
41 38.82532419862557
42 35.77744566541545
43 33.00369551447916
44 30.473481547463575
45 28.163259763237637
46 26.05116053814255
47

## torch MLP

### np to tensor

In [18]:
N,D_in,H,D_out=64,1000,100,10
# N:features H:hidden x*w1 =>H H*w2=>D_out
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)
w1 = torch.randn(D_in,H)
w2 = torch.randn(H,D_out)
learning_rate = 1e-6
for i in range(500):
    # forward pass
    h = x.mm(w1)  # mm == dot
    h_relu = h.clamp(min=0) # 限制下限
    y_pred = h_relu.mm(w2) #(N,D_out)
    
    # loss fun
    loss = (y_pred-y).pow(2).sum().item()#.item() to array
    print(i,loss)
    
    #backward pass gradient
    grad_y_pred = 2.0*(y_pred-y) #loss 求导
    grad_w2 = h_relu.t().mm(grad_y_pred) # h_relu(局部梯度) * grad_y_pred(上游梯度)
    grad_h_relu = grad_y_pred.mm(w2.t()) # w2(局部梯度) * grad_y_pred(上游梯度)
    grad_h = grad_h_relu.clone()
    grad_h[h<0]=0
    grad_w1 = x.t().mm(grad_h) # x(局部梯度) * grad_h(上游梯度)
    #backward pass update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    

0 29927648.0
1 25241944.0
2 24215970.0
3 23274462.0
4 20461000.0
5 15847380.0
6 10785526.0
7 6743838.0
8 4097066.75
9 2563374.75
10 1707951.375
11 1225322.875
12 937441.625
13 752075.5625
14 623005.5
15 526967.9375
16 451907.0
17 391155.25
18 340815.40625
19 298573.75
20 262670.03125
21 231887.9375
22 205453.875
23 182670.609375
24 162847.5
25 145535.953125
26 130388.484375
27 117095.78125
28 105369.515625
29 95005.15625
30 85809.5390625
31 77629.5546875
32 70337.671875
33 63825.2421875
34 58000.91796875
35 52775.84375
36 48080.203125
37 43859.83203125
38 40057.50390625
39 36623.12890625
40 33516.09765625
41 30703.1484375
42 28150.02734375
43 25831.1953125
44 23723.888671875
45 21809.7890625
46 20065.087890625
47 18476.890625
48 17026.87890625
49 15700.634765625
50 14486.9150390625
51 13375.68359375
52 12357.0556640625
53 11422.76171875
54 10566.1083984375
55 9778.677734375
56 9054.5966796875
57 8388.59765625
58 7775.298828125
59 7209.61474609375
60 6688.0244140625
61 6206.798828125
62

In [21]:
x = torch.tensor(1.0,requires_grad=True)
w = torch.tensor(2.0,requires_grad=True)
b = torch.tensor(3.0,requires_grad=True)
y = w*x+b
y.backward()
w.grad
x.grad
# b.grad

tensor(2.)

### auto gradient

In [66]:
N,D_in,H,D_out=64,1000,100,10
# N:features H:hidden x*w1 =>H H*w2=>D_out
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)
w1 = torch.randn(D_in,H,requires_grad=True)
w2 = torch.randn(H,D_out,requires_grad=True)
learning_rate = 1e-6
for i in range(500):
    # forward pass
#     h = x.mm(w1)  # mm == dot
#     h_relu = h.clamp(min=0) #
#     y_pred = h_relu.mm(w2) #(N,D_out)
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    # loss fun
    loss = (y_pred-y).pow(2).sum()
    print(i,loss.item())#.item() to array
    
    #backward pass gradient

    loss.backward()
#     grad_y_pred = 2.0*(y_pred-y) #loss 求导
#     grad_w2 = h_relu.t().mm(grad_y_pred) # h_relu(局部梯度) * grad_y_pred(上游梯度)
#     grad_h_relu = grad_y_pred.mm(w2.t()) # w2(局部梯度) * grad_y_pred(上游梯度)
#     grad_h = grad_h_relu.clone()
#     grad_h[h<0]=0
#     grad_w1 = x.t().mm(grad_h) # x(局部梯度) * grad_h(上游梯度)

    #backward pass update weights
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

0 31060424.0
1 27335330.0
2 27355924.0
3 26683084.0
4 23116724.0
5 16956402.0
6 10706482.0
7 6180859.5
8 3537396.5
9 2149052.75
10 1432562.75
11 1045008.1875
12 816611.5
13 667652.25
14 561317.5625
15 480045.9375
16 415106.8125
17 361722.25
18 317068.3125
19 279211.875
20 246859.1875
21 219078.1875
22 195016.046875
23 174077.53125
24 155777.75
25 139739.5
26 125627.171875
27 113171.078125
28 102155.25
29 92389.578125
30 83698.578125
31 75954.015625
32 69035.3984375
33 62834.546875
34 57267.93359375
35 52267.09765625
36 47765.72265625
37 43701.29296875
38 40028.140625
39 36703.3359375
40 33688.765625
41 30951.9453125
42 28464.90234375
43 26198.671875
44 24133.4140625
45 22249.806640625
46 20529.740234375
47 18957.154296875
48 17518.517578125
49 16201.1728515625
50 14993.69140625
51 13885.6337890625
52 12867.98828125
53 11932.564453125
54 11072.6591796875
55 10281.158203125
56 9552.119140625
57 8880.578125
58 8261.1298828125
59 7689.0234375
60 7160.6982421875
61 6672.0322265625
62 6219.7

### nn

In [23]:
N,D_in,H,D_out=64,1000,100,10
# N:features H:hidden x*w1 =>H H*w2=>D_out
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H,bias=False), # w_1 * x + b_1  #W的维度（1000，100）
    torch.nn.ReLU(), #(64,100)
    torch.nn.Linear(H,D_out,bias=False) #(64,10)
)
# w1 = torch.randn(D_in,H,requires_grad=True)
# w2 = torch.randn(H,D_out,requires_grad=True)
print(model)
torch.nn.init.normal_(model[0].weight) # 参数初始化为normal
torch.nn.init.normal_(model[2].weight) # model[1] 是relu 

loss_fn = torch.nn.MSELoss(reduction = "sum")
learning_rate = 1e-4

for i in range(500):
    # forward pass
#     h = x.mm(w1)  # mm == dot
#     h_relu = h.clamp(min=0) #
#     y_pred = h_relu.mm(w2) #(N,D_out)
#  #   y_pred = x.mm(w1).clamp(min=0).mm(w2)
    y_pred = model(x)
    # loss fun
    
    loss =loss_fn(y_pred , y)
#     loss = (y_pred-y).pow(2).sum()
    print(i,loss.item())#.item() to array
    
    #backward pass gradient
    loss.backward()
#     grad_y_pred = 2.0*(y_pred-y) #loss 求导
#     grad_w2 = h_relu.t().mm(grad_y_pred) # h_relu(局部梯度) * grad_y_pred(上游梯度)
#     grad_h_relu = grad_y_pred.mm(w2.t()) # w2(局部梯度) * grad_y_pred(上游梯度)
#     grad_h = grad_h_relu.clone()
#     grad_h[h<0]=0
#     grad_w1 = x.t().mm(grad_h) # x(局部梯度) * grad_h(上游梯度)

    #backward pass update weights
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
#         w1 -= learning_rate * w1.grad
#         w2 -= learning_rate * w2.grad
#         w1.grad.zero_()
#         w2.grad.zero_()
    model.zero_grad()


Sequential(
  (0): Linear(in_features=1000, out_features=100, bias=False)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=False)
)
0 31266974.0
1 1299046334464.0
2 1.5610677076213187e+25
3 inf
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
24 nan
25 nan
26 nan
27 nan
28 nan
29 nan
30 nan
31 nan
32 nan
33 nan
34 nan
35 nan
36 nan
37 nan
38 nan
39 nan
40 nan
41 nan
42 nan
43 nan
44 nan
45 nan
46 nan
47 nan
48 nan
49 nan
50 nan
51 nan
52 nan
53 nan
54 nan
55 nan
56 nan
57 nan
58 nan
59 nan
60 nan
61 nan
62 nan
63 nan
64 nan
65 nan
66 nan
67 nan
68 nan
69 nan
70 nan
71 nan
72 nan
73 nan
74 nan
75 nan
76 nan
77 nan
78 nan
79 nan
80 nan
81 nan
82 nan
83 nan
84 nan
85 nan
86 nan
87 nan
88 nan
89 nan
90 nan
91 nan
92 nan
93 nan
94 nan
95 nan
96 nan
97 nan
98 nan
99 nan
100 nan
101 nan
102 nan
103 nan
104 nan
105 nan
106 nan
107 nan
108 nan
109 nan
110 nan
111 nan
112 nan
113 nan
114 nan
11

In [77]:
model[0]
model[0].weight

Parameter containing:
tensor([[ 0.0297, -0.0223,  0.0059,  ..., -0.0043,  0.0226, -0.0016],
        [ 0.0055, -0.0272,  0.0309,  ...,  0.0287, -0.0164,  0.0209],
        [-0.0284, -0.0252,  0.0082,  ...,  0.0122,  0.0176, -0.0233],
        ...,
        [ 0.0144,  0.0240,  0.0193,  ..., -0.0232,  0.0083, -0.0237],
        [ 0.0031, -0.0233,  0.0084,  ...,  0.0190,  0.0296, -0.0041],
        [-0.0146,  0.0087, -0.0169,  ...,  0.0126, -0.0005, -0.0194]],
       requires_grad=True)

### 参数优化

In [25]:
N,D_in,H,D_out=64,1000,100,10
# N:features H:hidden x*w1 =>H H*w2=>D_out
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H,bias=False), # w_1 * x + b_1 
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out,bias=False)
)

loss_fn = torch.nn.MSELoss(reduction = "sum")
# learning_rate = 1e-4
# optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

torch.nn.init.normal_(model[0].weight) # 参数初始化为normal
torch.nn.init.normal_(model[2].weight)
learning_rate = 1e-6
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

for i in range(500):
    y_pred = model(x) # model.forward(x)
    # loss fun
    loss =loss_fn(y_pred , y)
    print(i,loss.item())#.item() to array
    optimizer.zero_grad()
    #backward pass gradient
    loss.backward()
    optimizer.step()

0 25870464.0
1 21324600.0
2 19242006.0
3 17442222.0
4 14945994.0
5 11874903.0
6 8704280.0
7 6034331.5
8 4048076.0
9 2715549.75
10 1858900.5
11 1320676.625
12 977948.25
13 754441.0
14 602621.125
15 495005.6875
16 415311.96875
17 353985.8125
18 305194.375
19 265444.65625
20 232404.328125
21 204524.0
22 180730.703125
23 160288.140625
24 142592.421875
25 127194.6953125
26 113741.0859375
27 101943.9765625
28 91562.7421875
29 82406.296875
30 74297.9140625
31 67105.1328125
32 60710.25
33 55015.01953125
34 49924.734375
35 45374.1875
36 41298.7265625
37 37639.2734375
38 34347.46875
39 31381.09375
40 28704.7578125
41 26286.19140625
42 24097.361328125
43 22115.1015625
44 20315.53515625
45 18680.09375
46 17193.498046875
47 15840.732421875
48 14607.984375
49 13483.10546875
50 12455.083984375
51 11514.6533203125
52 10653.3935546875
53 9864.041015625
54 9139.7060546875
55 8474.7109375
56 7863.3603515625
57 7300.890625
58 6783.2490234375
59 6306.25
60 5866.3251953125
61 5460.31884765625
62 5085.577148

In [89]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self,D_in,H,D_out):
        super(TwoLayerNet,self).__init__()
        self.linear1 =  torch.nn.Linear(D_in,H,bias=False)
        self.linear2 =  torch.nn.Linear(H,D_out,bias=False)
        
    def forward(self,x):
        y_pred=self.linear2(self.linear1(x).clamp(min=0))
        return y_pred
    
        
N,D_in,H,D_out=64,1000,100,10
# N:features H:hidden x*w1 =>H H*w2=>D_out
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)


model = TwoLayerNet(D_in,H,D_out)
# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in,H,bias=False), # w_1 * x + b_1 
#     torch.nn.ReLU(),
#     torch.nn.Linear(H,D_out,bias=False)
# )


loss_fn = torch.nn.MSELoss(reduction = "sum")
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


for i in range(500):
    y_pred = model(x) 
    # loss fun
    
    loss =loss_fn(y_pred , y)
    print(i,loss.item())#.item() to array
    optimizer.zero_grad()
    #backward pass gradient
    loss.backward()
    optimizer.step()
    


0 677.0870971679688
1 660.4974975585938
2 644.332275390625
3 628.5753173828125
4 613.2144775390625
5 598.2674560546875
6 583.6878662109375
7 569.4609375
8 555.627197265625
9 542.1997680664062
10 529.1843872070312
11 516.5291748046875
12 504.1893005371094
13 492.26495361328125
14 480.7138977050781
15 469.5166015625
16 458.61175537109375
17 447.99261474609375
18 437.6741638183594
19 427.6448669433594
20 417.8483581542969
21 408.2877502441406
22 398.9918212890625
23 389.95751953125
24 381.15020751953125
25 372.53326416015625
26 364.1355895996094
27 355.91497802734375
28 347.9114685058594
29 340.08001708984375
30 332.4192810058594
31 324.9525146484375
32 317.7039794921875
33 310.63250732421875
34 303.72113037109375
35 296.9505310058594
36 290.3032531738281
37 283.80389404296875
38 277.4469909667969
39 271.21453857421875
40 265.09869384765625
41 259.0865478515625
42 253.1988067626953
43 247.41407775878906
44 241.74472045898438
45 236.17225646972656
46 230.70086669921875
47 225.3445739746093